In [8]:
#NEED TO BE RUN TWICE (Unknown error)
import numpy as np
import pandas as pd
import pickle
import os
import random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy.stats as stats
import xarray as xr
import zipfile
from dateutil.relativedelta import relativedelta
import scipy.stats as stats
from scipy.optimize import minimize
import plotly.graph_objs as go
import pyreadr
import math
import time
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "zhippofficiel"

In [9]:
# Statistical models and function for modelization of precipitations.
# Include GEV and EGPD functions.

def H(xi,X):
    X=np.array(X)
    return((1+xi*X)**(-1/xi))

def F(params,X):
    kappa,sigma,xi = params
    X=np.array(X)
    return(H(xi,X/sigma)**kappa)

def f(params,X):
    kappa,sigma,xi = params
    return((kappa/sigma)*((1+((xi/sigma)*X))**(-1-(kappa/xi))))

def minus_l_f(params,X):
    kappa,sigma,xi = params
    return -np.sum(list(np.log((kappa/sigma)*((1+((xi/sigma)*X))**(-1-(kappa/xi))))))

def H_inv(xi,X):
    X=np.array(X)
    return(((X**(-xi))-1)/xi)

def Q(params,p):
    kappa,sigma,xi = params
    p = np.array(p)
    return((sigma/xi)*((p**(-xi/kappa))-1))

def data_xhour(data,x):
    results = []
    N=len(data)
    for i in range(N//x):
        value = 0
        for j in range(x):
            value = value + data[i*x+j]
        results.append(value)
    return results

def get_optimal_params(lat,lon,duration,treshold): #lat,lon in '41.25' format; duration in hour (only int) ; treshold in m
    selected_value = list(full_dataset.sel(lon=lon, lat=lat).values)
    list_precip = np.array(selected_value)
    data_dur = np.array(data_xhour(list_precip,duration))
    cleaned_data = np.array([(x-treshold)*1000 for x in data_dur if x>treshold])
    initial_guess = [0.5, 1, 0.3]
    objective_function = lambda params: minus_l_f(params, cleaned_data)
    result_minimize = minimize(objective_function, initial_guess, method='Nelder-Mead')
    optimal_params = result_minimize.x
    return optimal_params

def file_oneD_all_points(duration,treshold):
    Store_params = []
    lons = list(full_dataset['lon'].values)
    lats = list(full_dataset['lat'].values)
    for lat in lats:
        print('Starting to work on points with latitude {}°N'.format(float(lat)))
        for lon in lons:
            optimal_params  = get_optimal_params(lat,lon,duration,treshold)
            res = list(optimal_params)
            res.append(lat)
            res.append(lon)
            Store_params.append(res)
    return Store_params

def create_save_optimal_params(durations,treshold): #durations in hours ([int] only) ; treshold in m 
    FILEDIR_KEY_OUT_S3 = "ExtremeValuesPrecipitations/ERA Analysis/Optimal_parameters"       
    print(" ------------------------- Starting searching optimal parameters. ------------------------- ")
    for duration in durations:
        start_time = time.time()
        print(" ------------------------- For a period of {} hours. ------------------------- ".format(duration))
        FILE_KEY_OUT_S3 = FILEDIR_KEY_OUT_S3+"/optimal_params_"+str(duration)+"_hours.txt"
        optimal_params_oneD = file_oneD_all_points(duration,treshold)
        optimal_params_oneD_list=[]
        for x in optimal_params_oneD:
            value = list(x)
            optimal_params_oneD_list.append(value)
            column_names = ['kappa', 'sigma', 'xi','lat','lon']
        Optimal_params_df = pd.DataFrame(optimal_params_oneD_list, columns=column_names)
        FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3
        with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
            Optimal_params_df.to_csv(file_out, sep='\t', index=False)
        print(' --- Saving File Completed. ---')
        end_time = time.time()
        elapsed_time = end_time - start_time
        print("Time taken to compute data: {:.2f} seconds".format(elapsed_time))
    return("Task completed successfully!")

def return_levels_to_quantiles(return_levels,duration): # return_level en années ; duration en heures 
    return_levels = np.array(return_levels)
    res = (np.array([(duration/(365*time*24)) for time in return_levels]))
    return res

def IDF_one_D(params,return_levels,duration):
    quantiles = return_levels_to_quantiles(return_levels,duration)
    Qvalues = Q(params,quantiles)
    return Qvalues

def get_return_levels_int_or_cumul(lat,lon,durations,return_levels,Int):
    FILEDIR_KEY_S3 = "ExtremeValuesPrecipitations/ERA Analysis/Optimal_parameters"      
    Return_Values = [] 
    for dur in durations:
        FILE_KEY_S3 = FILEDIR_KEY_S3 + '/optimal_params_'+str(dur)+'_hours.txt'
        FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
        with fs.open(FILE_PATH_S3, mode="rb") as file_in:
            df = pd.read_csv(file_in, sep='\t')
        selected_point = df[(df['lat'] == lat) & (df['lon'] == lon)]
        optimal_params = [float(selected_point['kappa']),float(selected_point['sigma']),float(selected_point['xi'])]
        Qvalues = IDF_one_D(optimal_params,return_levels,dur)
        Return_Values.append(Qvalues)
    plt.figure()
    for i in range(len(return_levels)):
        values = []
        for j in range(len(durations)):
            if Int:
                values.append(Return_Values[j][i]/durations[j])
            if not Int:
                values.append(Return_Values[j][i])
        plt.plot(durations,values,marker='s',label='Return levels over {} Year'.format(return_levels[i]))
    plt.xticks(durations)
    plt.xlim(0.6,15)
    #plt.xscale('log')
    #plt.yscale('log')
    plt.grid()
    #plt.legend()
    plt.xlabel('Durations (hours)')
    if Int:
        plt.ylabel('Intensity (mm/hour)')
    if not Int:
        plt.ylabel('Cumuls (mm)')
    plt.title('IDF Curve')
    plt.show()
    return Return_Values

In [ ]:
# Loading ERA data from database

FILEDIR_KEY_S3 = "ExtremeValuesPrecipitations/ERA DATA"
file_name1 = BUCKET + "/" + FILEDIR_KEY_S3 + '/test.Rdata'
with fs.open(file_name1, mode="rb") as file_in:
    ds1 = pyreadr.read_r(file_in)
dataframe1 = ds1['A']

file_name2 = BUCKET + "/" + FILEDIR_KEY_S3 +'/test2.Rdata'
with fs.open(file_name1, mode="rb") as file_in:
    ds2 = pyreadr.read_r(file_in)
dataframe2 = ds2['A']
file_name3 = BUCKET + "/" + FILEDIR_KEY_S3 +'/test3.Rdata'
with fs.open(file_name1, mode="rb") as file_in:
    ds3 = pyreadr.read_r(file_in)
dataframe3 = ds3['A']
file_name4 = BUCKET + "/" + FILEDIR_KEY_S3 +'/test4.Rdata'
with fs.open(file_name1, mode="rb") as file_in:
    ds4 = pyreadr.read_r(file_in)
dataframe4 = ds4['A']

datasets = [dataframe1,dataframe2,dataframe3,dataframe4]
concatenated_dataset = xr.concat(datasets, dim='dim_2')
new_names = {'dim_0': 'lon', 'dim_1': 'lat', 'dim_2': 'date'}
full_dataset = concatenated_dataset.rename(new_names)